In [58]:
import re
import pickle
import tomotopy as tp

from rdkit.Chem.inchi import MolFromInchi
from rdkit.Chem import MolToSmiles

from MS2LDA.utils import retrieve_spec4doc
from MS2LDA.modeling import extract_motifs, create_motif_spectra
from MS2LDA.motif_parser import load_m2m_folder

from MS2LDA.Add_On.Fingerprints.FP_annotation import annotate_motifs as calc_fingerprints

In [42]:
motif_features = extract_motifs(ms2lda, top_n=20)

In [43]:
motif_spectra = create_motif_spectra(motif_features)

normally I could just load the m2m folder, but there was a bug in the code that overwrote the motif_id. If only 20 paramters are selected, some Mass2Motifs consist only of losses and they can be valuable but with the current spec2vec this is not possible to predict them. Also a Mass2Motif object should then be used instead of the current spectrum object. Right now, these motifs are removed and therefore instead of 2000 Mass2Motifs only 1951 Mass2Motifs are returned. By enumerating over the Mass2Motifs and giving them a new id (this was the bug), the information Mass2Motif id information is lost in the m2m files. <br>
As long as the ms2lda.bin file is available all data can be correctly retrieved.

---

In [7]:
with open("DDA-Suspectlist-2000motifs/doc2spec_map.pkl", "rb") as f:
    doc2spec_map = pickle.load(f)

In [5]:
ms2lda = tp.LDAModel.load("DDA-Suspectlist-2000motifs/ms2lda.bin")

In [6]:
help(retrieve_spec4doc)

Help on function retrieve_spec4doc in module MS2LDA.utils:

retrieve_spec4doc(doc2spec_map, ms2lda, doc_id)
    Retrieves the orginal spectrum for a given document based on a hashmap
    
    ARGS:
        doc2spec_map (dict): hash-spectrum pairs
        ms2lda (tomotopy object): LDA model
        doc_id (int): id of document
    
    RETURNS:
        retrieved_spec: matchms spectrum object



In [20]:
n_docs = len(ms2lda.docs)
print("Number of spectra used for modeling MS2LDA: ", n_docs)

Number of spectra used for modeling MS2LDA:  85898


In [61]:
def calc_tanimoto(fp1, fp2):
    fp1 = np.asarray(fp1, dtype=bool)
    fp2 = np.asarray(fp2, dtype=bool)

    intersection = np.logical_and(fp1, fp2).sum()
    union = np.logical_or(fp1, fp2).sum()

    return intersection / union

In [62]:
def inchi2smiles(inchi):
    # get only inchi code
    inchi = re.search(r'\"\"\"(.*)\"\"', inchi).group(0)
    mol = MolFromInchi(inchi)
    if mol:
        smiles = MolToSmiles(mol)
        return smiles
    else:
        return None

In [55]:
for doc_id in range(n_docs):
    # retrieve distribution for document
    top_1_motif, top_2_motif = ms2lda.docs[doc_id].get_topics(2)
    
    top_1_motif_id, top_1_motif_score = top_1_motif
    top_2_motif_id, top_2_motif_score = top_2_motif

    # retrieve spectrum
    top_1_motif_spec = motif_spectra[top_1_motif_id]
    top_2_motif_spec = motif_spectra[top_2_motif_id]

    # extract document/input spectrum annotation
    input_spec = retrieve_spec4doc(doc2spec_map, ms2lda, doc_id)
    inchi = input_spec.get("inchi")
    smiles = inchi2smiles(inchi)

    # extract fingerprints
    #top_1_smiles = top_1_motif_spec.get("Auto_annotation")
    #top_1_motif_fps = calc_fingerprints([top_1_smiles])
    print()
    if doc_id == 50:
        break

{'charge': 1, 'ionmode': 'positive', 'inchi': '"""InChI=1S/C26H43NO5/c1-15(4-9-23(30)27-14-24(31)32)19-7-8-20-18-6-5-16-12-17(28)10-11-25(16,2)21(18)13-22(29)26(19,20)3/h15-22,28-29H,4-14H2,1-3H3,(H,27,30)(H,31,32)/t15-,16-,17-,18+,19-,20+,21+,22-,25+,26-/m1/s1"" - 273.282 Da"', 'scans': '1', 'ms_level': '2', 'instrument_type': 'ESI-qTof', 'file_name': 'suspect_list_batch_creation.mgf', 'peptide_sequence': '*..*', 'organism_name': 'GNPS-SUSPECTLIST', 'compound_name': '"Suspect related to ""((4R)-4-((3R,5R,9S,10S,12R,13R,14S,17R)-3,12-dihydroxy-10,13-dimethylhexadecahydro-1H-cyclopenta[a]phenanthren-17-yl)pentanoyl)glycine"" (predicted molecular formula SIRIUS: unknown / BUDDY: unknown) with delta m/z -273.282 (putative explanation: unspecified; atomic difference: unspecified)', 'principal_investigator': 'Pieter Dorrestein', 'data_collector': 'Wout Bittremieux', 'submit_user': 'jzemlin', 'confidence': '4', 'spectrum_id': 'CCMSLIB00012079748', 'precursor_mz': 626.351, 'adduct': '[2M+H]+"

In [52]:
top_1_motif_spec.metadata

{'id': 'motif_908', 'charge': 1, 'ms2accuracy': 0.005, 'motifset': 'unknown'}